# Part 1
## Building the Data Frame from Wikipedia Page

In [1]:
import numpy as np
import pandas as pd

In [2]:
toronto_postal_df = pd.read_html(io='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Postal')[0]
toronto_postal_df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [3]:
toronto_postal_df.replace('Not assigned', np.NaN, inplace = True)

# Replace Neighborhood values with Borough values, when Neighborhood *IS* NaN
# If the Borough value is NaN, Neighborhood remains NaN, if the Borough has valid value, Neighborhood takes on value
toronto_postal_df['Neighborhood'].where(toronto_postal_df['Neighborhood'].notna(),toronto_postal_df['Borough'], inplace = True)

# Drop remaining NaN's 
toronto_postal_df.dropna(inplace = True)

toronto_postal_df.reset_index(drop = True, inplace = True)

In [4]:
toronto_postal_df.shape

(103, 3)

# Part 2
## Appending Data Frame with Latitude and Longitude of Zip Codes

In [62]:
import geocoder as geoc


In [14]:
import urllib.request
url = 'https://cocl.us/Geospatial_data'
filename = 'toronto_data.csv'
urllib.request.urlretrieve(url, filename)

('toronto_data.csv', <http.client.HTTPMessage at 0x15c9a932d90>)

In [5]:
# Using Provided CSV for simplicity
ll_df = pd.read_csv('toronto_data.csv')
ll_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
toronto_zip_df = toronto_postal_df.join(ll_df.set_index('Postal Code'), on='Postal Code')
toronto_zip_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3
## Exploring and Clustoring Toronto Neighborhoods

In [7]:
# dotenv is used to secure and hide API credentials
%load_ext dotenv

In [8]:
%dotenv

In [9]:
import folium
import requests

!pip install -U sklearn
from sklearn.cluster import KMeans

!pip install -U matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# Importing API Credentials
import os
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
VERSION = '20180323'

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: sklearn in c:\users\harry\appdata\roaming\python\python38\site-packages (0.0)
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: matplotlib in c:\users\harry\appdata\roaming\python\python38\site-packages (3.3.0)


In [12]:
# Splitting out "Toronto" named boroughs
toronto_only_df = toronto_zip_df[toronto_zip_df['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_only_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [13]:
# Latitude and Longitude values from google
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_only_df['Latitude'], toronto_only_df['Longitude'], toronto_only_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [14]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names = toronto_only_df['Neighborhood'],
                                latitudes = toronto_only_df['Latitude'],
                                longitudes = toronto_only_df['Longitude']
                                )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [16]:
toronto_venues[toronto_venues['Neighborhood'] == 'Regent Park, Harbourfront']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
...,...,...,...,...,...,...,...
95,"Regent Park, Harbourfront",43.65426,-79.360636,Sultan Of Samosas,43.660227,-79.366226,Indian Restaurant
96,"Regent Park, Harbourfront",43.65426,-79.360636,Regent Park,43.660383,-79.361810,Park
97,"Regent Park, Harbourfront",43.65426,-79.360636,King's Place647-352-0786,43.658278,-79.370925,Pakistani Restaurant
98,"Regent Park, Harbourfront",43.65426,-79.360636,Tim Hortons,43.659935,-79.361306,Coffee Shop


In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",48,48,48,48,48,48
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
toronto_onehot.shape

(3187, 275)

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.020833,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.066667,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.01,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.020000
5,Christie,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.000000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.010000,0.01,0.00,0.010000,0.00,0.010000,0.000000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.010000,0.00,0.00,0.010000,0.00,0.010000,0.000000,0.020000


In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2                Hotel  0.06
3           Restaurant  0.05
4  Japanese Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.08
1  Coffee Shop  0.07
2          Bar  0.05
3   Restaurant  0.05
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0                  Park  0.10
1           Coffee Shop  0.08
2               Brewery  0.06
3           Pizza Place  0.06
4  Fast Food Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Harbor / Marina  0.13
1      Coffee Shop  0.13
2             Café  0.13
3             Park  0.07
4   Scenic Lookout  0.07


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.09
1   

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Park,Beer Bar,Gym,Bakery,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Thrift / Vintage Store,Soccer Stadium
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Pizza Place,Brewery,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,French Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Harbor / Marina,Dog Run,Sculpture Garden,Sushi Restaurant,Park,Track,Dance Studio,Garden
4,Central Bay Street,Coffee Shop,Café,Clothing Store,Park,Yoga Studio,Bubble Tea Shop,Ramen Restaurant,Plaza,Pizza Place,Cosmetics Shop


In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 4, 3, 0, 0, 0, 2, 0, 0])

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_only_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Pub,Theater,Park,Diner,Restaurant,Bakery,Breakfast Spot,Sushi Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Pizza Place,Park,Italian Restaurant,Café,Gay Bar,Ice Cream Shop,Japanese Restaurant,Diner,Middle Eastern Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Gastropub,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Pizza Place,Plaza,Theater,Poke Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Clothing Store,Italian Restaurant,Seafood Restaurant,Cosmetics Shop,Hotel,Pizza Place,Bookstore
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pizza Place,Pub,Japanese Restaurant,Breakfast Spot,Beach,Park,Burger Joint,Sandwich Place,Bakery


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters